In [1]:
import MySQLdb
import sqlalchemy
import pandas as pd
import numpy as np

In [103]:
data = pd.read_csv("data/dados_jogadores.csv", sep=";", encoding="latin-1")

In [104]:
data.head()

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,Dama,Chile,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,Jogo da velha,Chile,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,Jogo da velha,Brasil,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,Jogo da velha,Argentina,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,Dama,Uruguai,1.0,54.0,55.0


In [105]:
data.isnull().sum()

jogador            0
genero             0
data_nascimento    0
jogo               0
pais               0
num_vitorias       8
num_derrotas       5
total_partidas     8
dtype: int64

In [106]:
data.dropna(inplace=True)

In [107]:
data.isnull().sum()

jogador            0
genero             0
data_nascimento    0
jogo               0
pais               0
num_vitorias       0
num_derrotas       0
total_partidas     0
dtype: int64

In [109]:
user = 'root'
password = ''
host = 'localhost'
database = 'dados_jogadores'

string_conexao = f'mysql://{user}:{password}@{host}/{database}'
string_conexao

'mysql://root:@localhost/dados_jogadores'

In [110]:
engine = sqlalchemy.create_engine(string_conexao)
conn = engine.connect()

In [112]:
pais = list(data['pais'].unique())
pais

['Chile', 'Brasil', 'Argentina', 'Uruguai', 'Paraguai']

In [113]:
jogo = list(data['jogo'].unique())
jogo

['Dama', 'Jogo da velha', 'Xadrez']

In [114]:
data.shape

(478, 8)

In [117]:
for p in pais:
    try:
        query = f"""insert into pais (nome_pais)
        values ('{p}')"""
        conn.execute(query)
        print(f"Registro inserido com sucesso: {p}")
    except Exception as e:
        print(f"Não foi possível inserir o registro {p}. O erro foi {e}")

Registro inserido com sucesso: Chile
Registro inserido com sucesso: Brasil
Registro inserido com sucesso: Argentina
Registro inserido com sucesso: Uruguai
Registro inserido com sucesso: Paraguai


In [118]:
for j in jogo:
    try:
        query = f"""insert into jogo (nome_jogo)
        values ('{j}')"""
        conn.execute(query)
        print(f"Registro inserido com sucesso: {j}")
    except Exception as e:
        print(f"Não foi possível inserir o registro {j}. O erro foi {e}")

Registro inserido com sucesso: Dama
Registro inserido com sucesso: Jogo da velha
Registro inserido com sucesso: Xadrez


In [119]:
df_jogador = data[['jogador', 'genero', 'data_nascimento', 'jogo', 'pais', 'num_vitorias', 'num_derrotas', 'total_partidas']]
df_jogador

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,Dama,Chile,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,Jogo da velha,Chile,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,Jogo da velha,Brasil,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,Jogo da velha,Argentina,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,Dama,Uruguai,1.0,54.0,55.0
...,...,...,...,...,...,...,...,...
493,Jogador 494,Feminino,07/11/1977,Dama,Uruguai,31.0,11.0,42.0
494,Jogador 495,Feminino,03/06/1974,Dama,Chile,52.0,28.0,80.0
495,Jogador 496,Masculino,09/02/1959,Dama,Chile,13.0,51.0,64.0
497,Jogador 498,Masculino,28/09/1958,Jogo da velha,Chile,5.0,3.0,8.0


In [120]:
df_pais = pd.read_sql('pais', con=conn)

In [121]:
df_pais

,cod_pais,nome_pais
0,1,Chile
1,2,Brasil
2,3,Argentina
3,4,Uruguai
4,5,Paraguai


In [123]:
dict_pais = df_pais.set_index(df_pais['nome_pais'])['cod_pais'].to_dict()
dict_pais

{'Chile': 1, 'Brasil': 2, 'Argentina': 3, 'Uruguai': 4, 'Paraguai': 5}

In [124]:
df_jogo = pd.read_sql('jogo', con=conn)
dict_jogo = df_jogo.set_index(df_jogo['nome_jogo'])['cod_jogo'].to_dict()
dict_jogo

{'Dama': 1, 'Jogo da velha': 2, 'Xadrez': 3}

In [125]:
df_jogador['pais'] = df_jogador['pais'].map(dict_pais)
df_jogador

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,Dama,1,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,Jogo da velha,1,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,Jogo da velha,2,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,Jogo da velha,3,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,Dama,4,1.0,54.0,55.0
...,...,...,...,...,...,...,...,...
493,Jogador 494,Feminino,07/11/1977,Dama,4,31.0,11.0,42.0
494,Jogador 495,Feminino,03/06/1974,Dama,1,52.0,28.0,80.0
495,Jogador 496,Masculino,09/02/1959,Dama,1,13.0,51.0,64.0
497,Jogador 498,Masculino,28/09/1958,Jogo da velha,1,5.0,3.0,8.0


In [126]:
df_jogador['jogo'] = df_jogador['jogo'].map(dict_jogo)
df_jogador

,jogador,genero,data_nascimento,jogo,pais,num_vitorias,num_derrotas,total_partidas
0,jogador 1,Masculino,03/09/1990,1,1,14.0,0.0,14.0
1,Jogador 2,Feminino,01/05/1986,2,1,39.0,20.0,59.0
2,Jogador 3,Masculino,04/10/1964,2,2,26.0,3.0,29.0
3,Jogador 4,Masculino,28/09/1979,2,3,9.0,72.0,81.0
4,Jogador 5,Masculino,03/09/1978,1,4,1.0,54.0,55.0
...,...,...,...,...,...,...,...,...
493,Jogador 494,Feminino,07/11/1977,1,4,31.0,11.0,42.0
494,Jogador 495,Feminino,03/06/1974,1,1,52.0,28.0,80.0
495,Jogador 496,Masculino,09/02/1959,1,1,13.0,51.0,64.0
497,Jogador 498,Masculino,28/09/1958,2,1,5.0,3.0,8.0


In [134]:
for j in df_jogador.itertuples():
    jogador = j.jogador
    genero = j.genero
    data_nasc = j.data_nascimento
    jogo = j.jogo
    pais = j.pais
    num_vit = j.num_vitorias
    num_der = j.num_derrotas
    total = j.total_partidas
    try:
        query = f"""insert into jogador (nome_jogador,genero, data_nascimento, num_vitorias, num_derrotas,
            total_partidas, cod_pais, cod_jogo)
        values ('{jogador}', '{genero}', '{data_nasc}', '{num_vit}', '{num_der}', '{total}', '{pais}', '{jogo}')"""
        conn.execute(query)
        print(f"Registro inserido com sucesso: {jogador}")
    except Exception as e:
        print(f"Não foi possível inserir o registro {jogador}. O erro foi {e}")

Registro inserido com sucesso: jogador 1
Registro inserido com sucesso: Jogador 2
Registro inserido com sucesso: Jogador 3
Registro inserido com sucesso: Jogador 4
Registro inserido com sucesso: Jogador 5
Registro inserido com sucesso: Jogador 6
Registro inserido com sucesso: Jogador 7
Registro inserido com sucesso: Jogador 8
Registro inserido com sucesso: Jogador 9
Registro inserido com sucesso: Jogador 10
Registro inserido com sucesso: Jogador 11
Registro inserido com sucesso: Jogador 12
Registro inserido com sucesso: Jogador 13
Registro inserido com sucesso: Jogador 14
Registro inserido com sucesso: Jogador 15
Registro inserido com sucesso: Jogador 16
Registro inserido com sucesso: Jogador 17
Registro inserido com sucesso: Jogador 18
Registro inserido com sucesso: Jogador 19
Registro inserido com sucesso: Jogador 20
Registro inserido com sucesso: Jogador 21
Registro inserido com sucesso: Jogador 22
Registro inserido com sucesso: Jogador 23
Registro inserido com sucesso: Jogador 24
R